# Entrega N° 5 - ProyectoDS_ParteIII


|Autor|Fecha|
|:----:|:----:|
|Francisco Tagliavini|2023-11-29|


## Motivo

Determinación de capacidad adquisitiva de las personas, ingreso anual

## Audiencia

Puede haber intereses respecto a nivel academico obtenido.
Para entidades de crédito de acuerdo a capacidad de pago.

## Contexto comercial

Brindar información de de capacidad de ingreso y crédito de las personas.
Para poder determinar si tiene capacidad de pago de deuda.

## OBJETIVO:

Se propone predecir mediante ML si una persona puede superar un cierto ingreso anual o no.
En este análisis el resultado presenta solo 2 opciones, siendo la variable categórica y dicotómica.['<50k' ; '>50k']
El dataset propuesto ya cuente con información de la variable a predecir.
En este escenario el MODELO propuesto es de CLASIFICACIÓN.

Asimismo, las categorías existentes del datasets nos permiten poder trabajar con algunas relaciones e intentar establecer si existe algun tipo de correlación más alta entre algunas de estas con otras variables ademas de la variable objetivo.

Para este tipo de análisis se puede utilizar el método de Hipótesis nula, en dónde proponemos una conclusión a la que debemos rechazar, por ejemplo:

* Las mujeres tienen ingrresos significativamente más bajos que los hombre
* Las persinas que trabajan menos horas tienen ingresos significativamente más bajos
* Las personas de raza ' Amer-Indian-Eskimo' tienen ingresos mas bajos
* Las personas de america latina tienen ingresos menores
* Las personas con mayores niveles de educación alcanzados son mayormente casados

Las condiciones de rechazo, se establecen a partir de un humbral de confiabibilidad, en donde se observa que la condicion de la Hipótesis nula queda por fuera del nivel de confiaza, esta es rechazada - indicando que no hay información suficiente en la muestra (o dataset) que permita validar la hipótesis planteada.

Para una primera aproximación, con un análisis bivariado mediante una gráfica de mapa de puntos (scatter plot) se prodría visualizar si existe algún tipo de relación.


## Interes por análisis del dataset con un método no supervisado [Parte II]

Se prodría analizar el DataSet sin el valor de la etiqueta, analizar mediante un sistema no supervisado y analizar mediate el método k-mains y el método de codos si el posible detetar más categorías.


# Cargo las librerias que voy a utilizar

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
!pip install mlxtend

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from sklearn.preprocessing import OrdinalEncoder
import sys
import joblib
sys.modules['sklearn.externals.joblib'] = joblib
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

import pandas.testing
import statsmodels.api as sm
import pandas.util.testing as tm

ModuleNotFoundError: No module named 'statsmodels'

In [75]:
#Clasificacion
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [83]:
#regresión
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

# Leo el dataset (archivo.csv)
* Me fijo si se lee bien con los primeros 3 registros

In [5]:
df = pd.read_csv('dataset.csv')
df.head(3)

,Unnamed: 0,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


# Analisis exploratorio de datos (EDA)

### Pruebo con el método info()
Resumen del dataset luego de la carga

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      48842 non-null  int64 
 1   age             48842 non-null  int64 
 2   workclass       47879 non-null  object
 3   fnlwgt          48842 non-null  int64 
 4   education       48842 non-null  object
 5   education-num   48842 non-null  int64 
 6   marital-status  48842 non-null  object
 7   occupation      47876 non-null  object
 8   relationship    48842 non-null  object
 9   race            48842 non-null  object
 10  sex             48842 non-null  object
 11  capital-gain    48842 non-null  int64 
 12  capital-loss    48842 non-null  int64 
 13  hours-per-week  48842 non-null  int64 
 14  native-country  48568 non-null  object
 15  income          48842 non-null  object
dtypes: int64(7), object(9)
memory usage: 6.0+ MB


# sigo analizando el tamaño del data set
Determino cunatos registros tiene el dataset y sus campos

In [7]:
df.shape

(48842, 16)

## Analizo los campos numericos media, mediana, min, max y percentiles
Analizo solo los campos que son numéricos para establecer las variaciones, dispersión y demas valores de tendencia central para tener una idea/detalle del campo

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,48842.0,24420.500000,14099.615261,0.0,12210.25,24420.5,36630.75,48841.0
age,48842.0,38.643585,13.710510,17.0,28.00,37.0,48.00,90.0
fnlwgt,48842.0,189664.134597,105604.025423,12285.0,117550.50,178144.5,237642.00,1490400.0
education-num,48842.0,10.078089,2.570973,1.0,9.00,10.0,12.00,16.0
capital-gain,48842.0,1079.067626,7452.019058,0.0,0.00,0.0,0.00,99999.0
capital-loss,48842.0,87.502314,403.004552,0.0,0.00,0.0,0.00,4356.0
hours-per-week,48842.0,40.422382,12.391444,1.0,40.00,40.0,45.00,99.0


## Veo los campos disponibles
Listo todos los nombres de las columnas

In [9]:
df.columns

Index(['Unnamed: 0', 'age', 'workclass', 'fnlwgt', 'education',
       'education-num', 'marital-status', 'occupation', 'relationship', 'race',
       'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
       'native-country', 'income'],
      dtype='object')

## Consulto por lo tipos de datos de los campos del dataset...
Veo que tipo de datos tiene cada campos del dataset

In [10]:
df.dtypes

Unnamed: 0         int64
age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
income            object
dtype: object

**Los campos numericos son todos de tipo Integer**

# Diccionario de campos y variables

* age numerico entero - puedo analizar la distribución de rango etarios de la población del dataset (74 categorias de edad)
* workclass: string (8 categorias) hay un valor no definido 
* fnlwgt: 
* education: string (16 categorias)
* education-num: numerico entero
* marital-status: string (7 categorias)
* occupation: string (15 categorias)
* relationship: string (6 categorias)
* race: string (6 categorias)
* sex: string (2 categorias)
* capital-gain:
* capital-loss:
* hours-per-week:
* native-country:
* income:

# Analizo valores repetidos o categorias de cada campo

In [11]:
df['age'].value_counts()

age
36    1348
35    1337
33    1335
23    1329
31    1325
      ... 
88       6
85       5
87       3
89       2
86       1
Name: count, Length: 74, dtype: int64

In [12]:
df['workclass'].value_counts()

workclass
Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
?                    1836
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: count, dtype: int64

In [13]:
df['education'].value_counts()

education
HS-grad         15784
Some-college    10878
Bachelors        8025
Masters          2657
Assoc-voc        2061
11th             1812
Assoc-acdm       1601
10th             1389
7th-8th           955
Prof-school       834
9th               756
12th              657
Doctorate         594
5th-6th           509
1st-4th           247
Preschool          83
Name: count, dtype: int64

In [14]:
df['education-num'].value_counts()

education-num
9     15784
10    10878
13     8025
14     2657
11     2061
7      1812
12     1601
6      1389
4       955
15      834
5       756
8       657
16      594
3       509
2       247
1        83
Name: count, dtype: int64

In [15]:
df['marital-status'].value_counts()

marital-status
Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: count, dtype: int64

In [16]:
df['occupation'].value_counts()

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
?                    1843
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

In [17]:
df['relationship'].value_counts()

relationship
Husband           19716
Not-in-family     12583
Own-child          7581
Unmarried          5125
Wife               2331
Other-relative     1506
Name: count, dtype: int64

In [18]:
df['race'].value_counts()

race
White                 41762
Black                  4685
Asian-Pac-Islander     1519
Amer-Indian-Eskimo      470
Other                   406
Name: count, dtype: int64

In [19]:
df['sex'].value_counts()

sex
Male      32650
Female    16192
Name: count, dtype: int64

In [20]:
df['capital-gain'].value_counts()

capital-gain
0        44807
15024      513
7688       410
7298       364
99999      244
         ...  
22040        1
2387         1
1639         1
1111         1
6612         1
Name: count, Length: 123, dtype: int64

In [21]:
df['capital-loss'].value_counts()

capital-loss
0       46560
1902      304
1977      253
1887      233
2415       72
        ...  
1539        1
1870        1
1911        1
2465        1
1421        1
Name: count, Length: 99, dtype: int64

|Variable Name|Role|Type|Demographic|Description|Units|Missing Values|
|:----|:----|:----|:----|:----|:----|:----|
|age|Feature|Integer|Age|N/A||no|
|workclass|Feature|Categorical|Income|Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked||yes|
|fnlwgt|Feature|Integer||||no|
|education|Feature|Categorical|Education Level|Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool||no|
|education-num|Feature|Integer|Education Level|||no|
|marital-status|Feature|Categorical|Other|Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse|||no|
|occupation|Feature|Categorical|Other|Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces|||yes|
|relationship|Feature|Categorical|Other|Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried|||no|
|race|Feature|Categorical|Race|White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black|||no|
|sex|Feature|Binary|Sex|Female, Male|||no|
|capital-gain|Feature|Integer||||no|
|capital-loss|Feature|Integer||||no|
|hours-per-week|Feature|Integer||||no|
|native-country|Feature|Categorical|Other|United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands|||yes|
|income|Target|Binary|Income|>50K, <=50K||no|

# vamos con los gráficos de MATPLOTLIB

In [ ]:
#Me fijo los estilos disponibles
import matplotlib as mpl
mpl.style.available

In [ ]:
#Selecciono un estilo
mpl.style.use('bmh')

## Análisis univariable

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.hist(df['age'])
ax.set_title('Distribucion de edades del dataset')
ax.set_xlabel('Edades')
ax.set_ylabel('Cantidad de registros')
ax.set(xlim=(0, 100), xticks=np.arange(0, 100,10),
       ylim=(0, 10000), yticks=np.arange(0, 10000,1000))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['age'])
ax.set_title('Distribucion de edades del dataset')
ax.set_xlabel('')
ax.set_ylabel('Edad')

plt.show()

observamos varios valores outliers que pueden correr el valor de la media...se debería analizar cuanto se corre la media y la mediana.
Considerar esto para el relleno de datos o valores NaN

Voy a repetir la grafica eliminando los outliers para limpiar un poco el ploteo

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['age'],showfliers=False)
ax.set_title('Distribucion de edades del dataset')
ax.set_xlabel('')
ax.set_ylabel('Edades')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.hist(df['hours-per-week'])
ax.set_title('Distribucion de horas laborales semanales')
ax.set_xlabel('Cantidad de registros')
ax.set_ylabel('')
ax.set(xlim=(0, 100), xticks=np.arange(0, 110,10),
       ylim=(0, 30000), yticks=np.arange(0, 33000,3000))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['hours-per-week'])
ax.set_title('Distribucion de horas laborales semanales')
ax.set_xlabel('Cantidad de registros')
ax.set_ylabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['hours-per-week'],showfliers=False)
ax.set_title('Distribucion de horas laborales semanales')
ax.set_xlabel('Cantidad de registros')
ax.set_ylabel('')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.hist(df['capital-gain'])
ax.set_title('Distribucion de ingresos anuales')
ax.set_xlabel('Ingresos (U$S)')
ax.set_ylabel('Cantidad de registros')
ax.set(xlim=(0, 100000), xticks=np.arange(0, 110000,10000),
       ylim=(0, 50000), yticks=np.arange(0, 55000,5000))
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['capital-gain'])
ax.set_title('Distribucion de ingresos anuales')
ax.set_xlabel('Ingresos (U$S)')
ax.set_ylabel('Cantidad de registros')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
ax.boxplot(df['capital-gain'],showfliers=False)
ax.set_title('Distribucion de ingresos anuales')
ax.set_xlabel('Ingresos (U$S)')
ax.set_ylabel('Cantidad de registros')

plt.show()

## Análisis multivariado

In [ ]:
test_1 = df.loc[df['sex'] == 'Female',['sex','capital-gain']]

x = df['sex']
y = df['capital-gain']

fig, ax = plt.subplots()

ax.scatter(x, y)

#ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#       ylim=(0, 8), yticks=np.arange(1, 8))

plt.show()

In [ ]:
x = df['capital-loss']
y = df['capital-gain']

fig, ax = plt.subplots()

ax.scatter(x, y)

#ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#       ylim=(0, 8), yticks=np.arange(1, 8))

plt.show()

In [ ]:
x = df['hours-per-week']
y = df['capital-gain']

fig, ax = plt.subplots()

ax.scatter(x, y)

#ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#       ylim=(0, 8), yticks=np.arange(1, 8))

plt.show()

# Data wrangling


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      48842 non-null  int64 
 1   age             48842 non-null  int64 
 2   workclass       47879 non-null  object
 3   fnlwgt          48842 non-null  int64 
 4   education       48842 non-null  object
 5   education-num   48842 non-null  int64 
 6   marital-status  48842 non-null  object
 7   occupation      47876 non-null  object
 8   relationship    48842 non-null  object
 9   race            48842 non-null  object
 10  sex             48842 non-null  object
 11  capital-gain    48842 non-null  int64 
 12  capital-loss    48842 non-null  int64 
 13  hours-per-week  48842 non-null  int64 
 14  native-country  48568 non-null  object
 15  income          48842 non-null  object
dtypes: int64(7), object(9)
memory usage: 6.0+ MB


### Hay algunas columnas que tienen menos cantidad de registros
2, 7, 14

## Reviso que campos tienen NaN y su cantidad

In [23]:
df.isna().sum()

Unnamed: 0          0
age                 0
workclass         963
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation        966
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    274
income              0
dtype: int64

### Confirmo que las columnas con menos registros tienen campos NaN

In [24]:
df.shape

(48842, 16)

In [25]:
sum(df.isna().sum())

2203

In [26]:
df[['workclass','occupation','native-country']].dtypes

workclass         object
occupation        object
native-country    object
dtype: object

## total de registros del dataset son más de 48.842 y encontramos  2.203 NaN
## Son campos del tipo objeto (string) no es posible reemplazarlos por otros medidas de tendencia central 

In [27]:
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked',
       nan], dtype=object)

In [28]:
df['occupation'].unique()

array(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
       'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair',
       'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
       'Tech-support', '?', 'Protective-serv', 'Armed-Forces',
       'Priv-house-serv', nan], dtype=object)

In [29]:
df['native-country'].unique()

array(['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico',
       'South', 'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany',
       'Iran', 'Philippines', 'Italy', 'Poland', 'Columbia', 'Cambodia',
       'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'China', 'Japan', 'Yugoslavia', 'Peru',
       'Outlying-US(Guam-USVI-etc)', 'Scotland', 'Trinadad&Tobago',
       'Greece', 'Nicaragua', 'Vietnam', 'Hong', 'Ireland', 'Hungary',
       'Holand-Netherlands', nan], dtype=object)

### Vemos que ademas de NaN también hay campos con '?' 
Vamos a reemplazar el valor por NaN

In [30]:
df['workclass'].size

48842

### Pruebo escribi un código para recorrer el campo y encontrar el caracter extraño '?'

In [31]:
j = 0
for i in enumerate(df['workclass']):
    #print(i)
    if i[1] == '?':
        #print('entro!')
        j += 1
    else:
        j += 0
print(j)

1836


### Funcionó! - Ahora intento crear una funcion
### Llamo a la función y la paso los parámetros del campo y el valor que estoy buscando

In [32]:
def cuento_raro1(serie1,valor):
    j = 0
    for i in enumerate(serie1):
        #print(i)
        if i[1] == valor:
            #print('entro!')
            j += 1
        else:
            j += 0
    return j

In [33]:
cuento_raro1(df['workclass'],'?')

1836

In [34]:
cuento_raro1(df['occupation'],'?')

1843

In [35]:
cuento_raro1(df['native-country'], '?')

583

## voy a reemplazar '?' por NaN porque podemos contar mejor los missing values

In [36]:
df = df.replace('?', np.nan)

In [37]:
sum(df.isna().sum())

6465

In [38]:
df.head()

,Unnamed: 0,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [39]:
df_sin_na = df

In [40]:
sum(df_sin_na.isna().sum())

6465

In [41]:
df.isna().sum()

Unnamed: 0           0
age                  0
workclass         2799
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     857
income               0
dtype: int64

In [42]:
df_sin_na.dropna(subset=['workclass','occupation','native-country'], inplace= False).count()

Unnamed: 0        45222
age               45222
workclass         45222
fnlwgt            45222
education         45222
education-num     45222
marital-status    45222
occupation        45222
relationship      45222
race              45222
sex               45222
capital-gain      45222
capital-loss      45222
hours-per-week    45222
native-country    45222
income            45222
dtype: int64

In [43]:
df_sin_na = df_sin_na.dropna(subset=['workclass','occupation','native-country'])

In [44]:
df_sin_na.shape

(45222, 16)

In [45]:
df.shape

(48842, 16)

In [46]:
48842-45222

3620

## Se eliminaron 3.620 registros, menos de los 6.465, se entiende que en algunos registros tenian esos campos con NaN

## analizando perdidas de datos en el datasets...

In [47]:
print(f'la perdida por eliminación de registros es de un : {3620/48842 * 100:.2f} %')
print(f'la perdida por eliminación de 3 columnas es de un : {3/16 * 100:.2f} %')

la perdida por eliminación de registros es de un : 7.41 %
la perdida por eliminación de 3 columnas es de un : 18.75 %


## De esta primera parte se perdió 7,41% de registros
## La eliminación de registros tiene un menor impacto que eliminar el campo completo 

# 3.- Método de Feature Selection
## Reduccion de la dimensionalidad

### Revision de corelaciones

In [48]:
df_sin_na.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      45222 non-null  int64 
 1   age             45222 non-null  int64 
 2   workclass       45222 non-null  object
 3   fnlwgt          45222 non-null  int64 
 4   education       45222 non-null  object
 5   education-num   45222 non-null  int64 
 6   marital-status  45222 non-null  object
 7   occupation      45222 non-null  object
 8   relationship    45222 non-null  object
 9   race            45222 non-null  object
 10  sex             45222 non-null  object
 11  capital-gain    45222 non-null  int64 
 12  capital-loss    45222 non-null  int64 
 13  hours-per-week  45222 non-null  int64 
 14  native-country  45222 non-null  object
 15  income          45222 non-null  object
dtypes: int64(7), object(9)
memory usage: 5.9+ MB


In [49]:
# elimino del dataset la variable objetivo
X = df_sin_na.drop('income', axis=1)
X = X.drop('Unnamed: 0',axis=1)

In [50]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45222 entries, 0 to 48841
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   fnlwgt          45222 non-null  int64 
 3   education       45222 non-null  object
 4   education-num   45222 non-null  int64 
 5   marital-status  45222 non-null  object
 6   occupation      45222 non-null  object
 7   relationship    45222 non-null  object
 8   race            45222 non-null  object
 9   sex             45222 non-null  object
 10  capital-gain    45222 non-null  int64 
 11  capital-loss    45222 non-null  int64 
 12  hours-per-week  45222 non-null  int64 
 13  native-country  45222 non-null  object
dtypes: int64(6), object(8)
memory usage: 5.2+ MB


In [51]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [53]:
encoder = OrdinalEncoder()

In [54]:
#Hago una clasificación numérica para la columna 'workclass' porque es un string y la convierto en categórica numérica

X['workclass_encoded'] = encoder.fit_transform(X[['workclass']])
X['education_encoded'] = encoder.fit_transform(X[['education']])
X['marital-status_encoder'] = encoder.fit_transform(X[['marital-status']])
X['occupation_encoder'] = encoder.fit_transform(X[['occupation']])
X['relationship_encoder'] = encoder.fit_transform(X[['relationship']])
X['race_encoder'] = encoder.fit_transform(X[['race']])
X['sex_encoder'] = encoder.fit_transform(X[['sex']])
X['native-country_encoder'] = encoder.fit_transform(X[['native-country']])

X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,workclass_encoded,education_encoded,marital-status_encoder,occupation_encoder,relationship_encoder,race_encoder,sex_encoder,native-country_encoder
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,...,40,United-States,5.0,9.0,4.0,0.0,1.0,4.0,1.0,38.0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,...,13,United-States,4.0,9.0,2.0,3.0,0.0,4.0,1.0,38.0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,...,40,United-States,2.0,11.0,0.0,5.0,1.0,4.0,1.0,38.0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,...,40,United-States,2.0,1.0,2.0,5.0,0.0,2.0,1.0,38.0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,...,40,Cuba,2.0,9.0,2.0,9.0,5.0,2.0,0.0,4.0


In [55]:
y = encoder.fit_transform(df_sin_na[['income']])
y

array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [3.]])

In [58]:
X_sin_str = X[['age','fnlwgt','capital-gain','capital-loss','hours-per-week','workclass_encoded',
              'education_encoded','marital-status_encoder','occupation_encoder','relationship_encoder','race_encoder',
             'sex_encoder','native-country_encoder']]
#parece que el education-num es igual a education_encoded

In [59]:
X_sin_str.head()

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,workclass_encoded,education_encoded,marital-status_encoder,occupation_encoder,relationship_encoder,race_encoder,sex_encoder,native-country_encoder
0,39,77516,2174,0,40,5.0,9.0,4.0,0.0,1.0,4.0,1.0,38.0
1,50,83311,0,0,13,4.0,9.0,2.0,3.0,0.0,4.0,1.0,38.0
2,38,215646,0,0,40,2.0,11.0,0.0,5.0,1.0,4.0,1.0,38.0
3,53,234721,0,0,40,2.0,1.0,2.0,5.0,0.0,2.0,1.0,38.0
4,28,338409,0,0,40,2.0,9.0,2.0,9.0,5.0,2.0,0.0,4.0


In [61]:
X_sin_str.describe().T

,count,mean,std,min,25%,50%,75%,max
age,45222.0,38.547941,13.217870,17.0,28.00,37.0,47.0,90.0
fnlwgt,45222.0,189734.734311,105639.195134,13492.0,117388.25,178316.0,237926.0,1490400.0
capital-gain,45222.0,1101.430344,7506.430084,0.0,0.00,0.0,0.0,99999.0
capital-loss,45222.0,88.595418,404.956092,0.0,0.00,0.0,0.0,4356.0
hours-per-week,45222.0,40.938017,12.007508,1.0,40.00,40.0,45.0,99.0
workclass_encoded,45222.0,2.204502,0.958122,0.0,2.00,2.0,2.0,6.0
education_encoded,45222.0,10.313321,3.817014,0.0,9.00,11.0,12.0,15.0
marital-status_encoder,45222.0,2.585180,1.500458,0.0,2.00,2.0,4.0,6.0
occupation_encoder,45222.0,5.969572,4.026400,0.0,2.00,6.0,9.0,13.0
relationship_encoder,45222.0,1.412697,1.597227,0.0,0.00,1.0,3.0,5.0


### Veo que quizas es necesario realizar una normalización en los campos (fnlwgt, capital-gain, capital-loss)

### Voy a verificar con la matriz de correlación sin realizar ninguna normalización 

In [62]:
corr_matrix = X_sin_str.corr()

print(corr_matrix)

                             age    fnlwgt  capital-gain  capital-loss  \
age                     1.000000 -0.075792      0.079683      0.059351   
fnlwgt                 -0.075792  1.000000     -0.004110     -0.004349   
capital-gain            0.079683 -0.004110      1.000000     -0.032102   
capital-loss            0.059351 -0.004349     -0.032102      1.000000   
hours-per-week          0.101992 -0.018679      0.083880      0.054195   
workclass_encoded       0.085892 -0.034970      0.034543      0.008233   
education_encoded      -0.003511 -0.025318      0.029889      0.016877   
marital-status_encoder -0.271623  0.033096     -0.042399     -0.034820   
occupation_encoder     -0.004474 -0.002036      0.019435      0.015267   
relationship_encoder   -0.247564  0.009909     -0.056818     -0.058425   
race_encoder            0.023776 -0.029188      0.014078      0.020955   
sex_encoder             0.082053  0.027240      0.047444      0.046457   
native-country_encoder -0.003180 -0.06

## parece que no hay una clara dependencia entre los campos...

## Voy a analizar esto mismo pero con las librerías

In [63]:
sfs = SFS(LinearRegression(),
          k_features=11,
          forward=True,
          floating=False,
          scoring = 'r2',
          cv = 0)

In [64]:
X1 = X_sin_str

In [65]:
sfs.fit(X1, y)
sfs.k_feature_names_     #Lista final de features

('age',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'workclass_encoded',
 'education_encoded',
 'marital-status_encoder',
 'occupation_encoder',
 'relationship_encoder',
 'race_encoder',
 'sex_encoder')

### Entiende que no se puede reducir ningún campo...

## Voy por la normalización a ver si cambia algo

In [66]:
from sklearn.preprocessing import MinMaxScaler

ms = MinMaxScaler()

X1_normal = ms.fit_transform(X1)

In [68]:
X1_normal

array([[0.30136986, 0.04335003, 0.02174022, ..., 1.        , 1.        ,
        0.95      ],
       [0.45205479, 0.04727376, 0.        , ..., 1.        , 1.        ,
        0.95      ],
       [0.28767123, 0.1368765 , 0.        , ..., 1.        , 1.        ,
        0.95      ],
       ...,
       [0.28767123, 0.24476203, 0.        , ..., 1.        , 1.        ,
        0.95      ],
       [0.36986301, 0.04766648, 0.05455055, ..., 0.25      , 1.        ,
        0.95      ],
       [0.24657534, 0.11419533, 0.        , ..., 1.        , 1.        ,
        0.95      ]])

In [70]:
columnas: np.array = X1.columns

In [71]:
X1_normal = pd.DataFrame(X1_normal, columns=[columnas])

In [72]:
X1_normal.head()

,age,fnlwgt,capital-gain,capital-loss,hours-per-week,workclass_encoded,education_encoded,marital-status_encoder,occupation_encoder,relationship_encoder,race_encoder,sex_encoder,native-country_encoder
0,0.301370,0.043350,0.02174,0.0,0.397959,0.833333,0.600000,0.666667,0.000000,0.2,1.0,1.0,0.95
1,0.452055,0.047274,0.00000,0.0,0.122449,0.666667,0.600000,0.333333,0.230769,0.0,1.0,1.0,0.95
2,0.287671,0.136877,0.00000,0.0,0.397959,0.333333,0.733333,0.000000,0.384615,0.2,1.0,1.0,0.95
3,0.493151,0.149792,0.00000,0.0,0.397959,0.333333,0.066667,0.333333,0.384615,0.0,0.5,1.0,0.95
4,0.150685,0.219998,0.00000,0.0,0.397959,0.333333,0.600000,0.333333,0.692308,1.0,0.5,0.0,0.10


In [73]:
corr_matrix_normal = X1_normal.corr()

print(corr_matrix_normal)

                             age    fnlwgt capital-gain capital-loss  \
age                     1.000000 -0.075792     0.079683     0.059351   
fnlwgt                 -0.075792  1.000000    -0.004110    -0.004349   
capital-gain            0.079683 -0.004110     1.000000    -0.032102   
capital-loss            0.059351 -0.004349    -0.032102     1.000000   
hours-per-week          0.101992 -0.018679     0.083880     0.054195   
workclass_encoded       0.085892 -0.034970     0.034543     0.008233   
education_encoded      -0.003511 -0.025318     0.029889     0.016877   
marital-status_encoder -0.271623  0.033096    -0.042399    -0.034820   
occupation_encoder     -0.004474 -0.002036     0.019435     0.015267   
relationship_encoder   -0.247564  0.009909    -0.056818    -0.058425   
race_encoder            0.023776 -0.029188     0.014078     0.020955   
sex_encoder             0.082053  0.027240     0.047444     0.046457   
native-country_encoder -0.003180 -0.061098     0.007643     0.00

In [74]:
sfs.fit(X1_normal, y)
sfs.k_feature_names_     #Lista final de features

(('age',),
 ('capital-gain',),
 ('capital-loss',),
 ('hours-per-week',),
 ('workclass_encoded',),
 ('education_encoded',),
 ('marital-status_encoder',),
 ('occupation_encoder',),
 ('relationship_encoder',),
 ('race_encoder',),
 ('sex_encoder',))

## No cambia nada luego de la normalización
## Hay que llevar todos los campos al modelo

### Clasificación

In [77]:
X1_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45222 entries, 0 to 45221
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   (age,)                     45222 non-null  float64
 1   (fnlwgt,)                  45222 non-null  float64
 2   (capital-gain,)            45222 non-null  float64
 3   (capital-loss,)            45222 non-null  float64
 4   (hours-per-week,)          45222 non-null  float64
 5   (workclass_encoded,)       45222 non-null  float64
 6   (education_encoded,)       45222 non-null  float64
 7   (marital-status_encoder,)  45222 non-null  float64
 8   (occupation_encoder,)      45222 non-null  float64
 9   (relationship_encoder,)    45222 non-null  float64
 10  (race_encoder,)            45222 non-null  float64
 11  (sex_encoder,)             45222 non-null  float64
 12  (native-country_encoder,)  45222 non-null  float64
dtypes: float64(13)
memory usage: 4.5 MB


In [78]:
# definimos matriz de diseño X y vector respuesta y
X_clasificacion = X1_normal
y_clasificacion = y
# Separamos en entrenamiento/test en razon 80/20 %
X_train, X_test, y_train, y_test = train_test_split(X_clasificacion,y_clasificacion,test_size=0.2, random_state=1)
# Creamos un modelo Random Forest con parametros por defect
modelo = RandomForestClassifier(random_state=1)
modelo.fit(X_train, y_train)
# Obtenemos las predicciones del modelo con X_test
preds = modelo.predict(X_test)

C:\ProgramData\miniconda3\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [84]:
plt.figure(figsize=(10,6))
metrics.plot_confusion_matrix(modelo, X_test, y_test, display_labels=['Negative', 'Positive'])

AttributeError: module 'sklearn.metrics' has no attribute 'plot_confusion_matrix'

<Figure size 1000x600 with 0 Axes>

In [80]:
confusion = metrics.confusion_matrix(y_test, preds)
confusion.ravel()

array([3673,  483,  265,   67, 1895,  234,  136,   33,  448,   93,  821,
        135,  262,   53,  385,   62], dtype=int64)

## Me dan multiples campos de la matriz de confusion

In [81]:
accuracy = metrics.accuracy_score(y_test, preds)
accuracy

0.5295743504698729

In [82]:
# Todas las metricas en uno
print(metrics.classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.59      0.82      0.68      4488
         1.0       0.27      0.10      0.15      2298
         2.0       0.51      0.55      0.53      1497
         3.0       0.21      0.08      0.12       762

    accuracy                           0.53      9045
   macro avg       0.39      0.39      0.37      9045
weighted avg       0.46      0.53      0.47      9045



### Regresión

In [86]:
X_regresion = X1_normal
y_regresion = y

In [87]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_regresion,y_regresion,test_size=0.2,random_state=2)
from sklearn.linear_model import LinearRegression
# crear el modelo
lr = LinearRegression()
# Ajustar el modelo con X_train y y_train
lr.fit(X_train,y_train)
# PRedecir con X_test
y_pred = lr.predict(X_test)

In [88]:
from sklearn.metrics import mean_absolute_error
print("MAE",mean_absolute_error(y_test,y_pred))

MAE 0.7599298595130058


In [89]:
from sklearn.metrics import mean_squared_error
print("MSE",mean_squared_error(y_test,y_pred))

MSE 0.8224044701377135


In [90]:
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))

RMSE 0.9068651885135482


In [91]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,y_pred)
print(r2)

0.1459488046493035


# 4.- Elegir un método de regresión o clasificación
## Para entrenar los datos

# 5.- Cálculos de métricas para validar el modelo

# 6.- Generar conclusiones